# Filtros lineales espaciales - caso pasabajas

<div class="alert alert-block alert-success">
<b>Resumen:</b>  En este notebook exploraremos los filtros pasabajas lineales en el dominio espacial, aplicándolos a imágenes y observando su efecto, especialmente en presencia de ruido Salt & Pepper.
</div>

***

In [ ]:
# Importar las bibliotecas necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. Lectura de la imagen de prueba

In [ ]:
# Leer la imagen en escala de grises
imagen_gray_1 = cv2.imread('test_pattern_blurring_orig.tif', cv2.IMREAD_GRAYSCALE)

# Verificar si la imagen se ha cargado correctamente
if imagen_gray_1 is None:
    print("Error: La imagen no se pudo cargar. Verifica la ruta y el nombre del archivo.")
else:
    # Mostrar la imagen original
    plt.figure(figsize=(4,4))
    plt.imshow(imagen_gray_1, cmap='gray')
    plt.title('Imagen Original')
    plt.axis('off')
    plt.show()

## 2. Filtro Pasabajas con un Box Kernel

### 2.1. Kernels 

In [ ]:
# Crear kernels de tipo box (promedio) con diferentes tamaños
w_1_a = np.ones((3, 3), dtype=np.float32) / 9    # Kernel 3x3
w_1_b = np.ones((11, 11), dtype=np.float32) / 121  # Kernel 11x11
w_1_c = np.ones((21, 21), dtype=np.float32) / 441  # Kernel 21x21

Visualización kernels

In [ ]:
# Visualizar el kernel Gaussian 21x21
plt.figure(figsize=(3,3))
plt.imshow(w_1_c, cmap='gray')
plt.title('Gaussian Kernel 21x21')
plt.colorbar()
plt.show()

### 2.2. Filtrado

In [ ]:
# Aplicar los filtros a la imagen
g_1_a = cv2.filter2D(imagen_gray_1, -1, w_1_a, borderType=cv2.BORDER_REPLICATE)
g_1_b = cv2.filter2D(imagen_gray_1, -1, w_1_b, borderType=cv2.BORDER_REPLICATE)
g_1_c = cv2.filter2D(imagen_gray_1, -1, w_1_c, borderType=cv2.BORDER_REPLICATE)

### 2.3. Resultado

In [ ]:
# Mostrar las imágenes filtradas
fig, axs = plt.subplots(1, 3, figsize=(18,6))

axs[0].imshow(g_1_a, cmap='gray')
axs[0].set_title('Filtro Box 3x3')
axs[0].axis('off')

axs[1].imshow(g_1_b, cmap='gray')
axs[1].set_title('Filtro Box 11x11')
axs[1].axis('off')

axs[2].imshow(g_1_c, cmap='gray')
axs[2].set_title('Filtro Box 21x21')
axs[2].axis('off')

plt.show()

## 3. Filtro Pasabajas con Gaussian Kernel

### 3.1. Kernels

In [ ]:
# Crear kernels Gaussianos con diferentes tamaños y sigma
w_2_a = cv2.getGaussianKernel(3, 1) * cv2.getGaussianKernel(3, 1).T    # 3x3, sigma=1
w_2_b = cv2.getGaussianKernel(11, 2) * cv2.getGaussianKernel(11, 2).T  # 11x11, sigma=2
w_2_c = cv2.getGaussianKernel(21, 3) * cv2.getGaussianKernel(21, 3).T  # 21x21, sigma=3

Visualización kernels

In [ ]:
# Visualizar el kernel Gaussian 21x21
plt.figure(figsize=(3,3))
plt.imshow(w_2_c, cmap='gray')
plt.title('Gaussian Kernel 21x21')
plt.colorbar()
plt.show()

### 3.2. Filtrado

In [ ]:
# Aplicar los filtros Gaussianos a la imagen
g_2_a = cv2.filter2D(imagen_gray_1, -1, w_2_a, borderType=cv2.BORDER_REPLICATE)
g_2_b = cv2.filter2D(imagen_gray_1, -1, w_2_b, borderType=cv2.BORDER_REPLICATE)
g_2_c = cv2.filter2D(imagen_gray_1, -1, w_2_c, borderType=cv2.BORDER_REPLICATE)

### 2.3. Resultado

In [ ]:
# Mostrar las imágenes filtradas
fig, axs = plt.subplots(1, 3, figsize=(18,6))

axs[0].imshow(g_2_a, cmap='gray')
axs[0].set_title('Filtro Gaussiano 3x3')
axs[0].axis('off')

axs[1].imshow(g_2_b, cmap='gray')
axs[1].set_title('Filtro Gaussiano 11x11')
axs[1].axis('off')

axs[2].imshow(g_2_c, cmap='gray')
axs[2].set_title('Filtro Gaussiano 21x21')
axs[2].axis('off')

plt.show()

## 3. Efecto del ruido Salt & Pepper a la Imagen

### 3.1. Función para añadir ruido Salt & Pepper

In [ ]:
def add_salt_and_pepper_noise(image, prob):
    """
    Añade ruido "salt & pepper" a una imagen.
    
    Parámetros:
    - image: Imagen en escala de grises (numpy.ndarray).
    - prob: Probabilidad de que un píxel sea reemplazado por ruido.
    
    Retorna:
    - Imagen con ruido "salt & pepper" añadida.
    """
    # Crear una copia de la imagen para no modificar la original
    noisy_image = image.copy()
    
    # Generar una matriz de números aleatorios
    random_matrix = np.random.rand(*image.shape)
    
    # Determinar dónde poner el 'pepper' (0)
    noisy_image[random_matrix < (prob / 2)] = 0
    
    # Determinar dónde poner el 'salt' (255)
    noisy_image[random_matrix > 1 - (prob / 2)] = 255
    
    return noisy_image

### 3.2. Generación de imágenes con ruido de baja y alta densidad

In [ ]:
# Ruido de baja densidad
prob_low = 0.0025
n_sp_low = add_salt_and_pepper_noise(imagen_gray_1, prob_low)

# Ruido de alta densidad
prob_high = 0.1
n_sp_high = add_salt_and_pepper_noise(imagen_gray_1, prob_high)

### 3.3. Visualización

In [ ]:
# Mostrar las imágenes con ruido
fig, axs = plt.subplots(1, 2, figsize=(12,6))

axs[0].imshow(n_sp_low, cmap='gray')
axs[0].set_title('Ruido Salt & Pepper Bajo')
axs[0].axis('off')

axs[1].imshow(n_sp_high, cmap='gray')
axs[1].set_title('Ruido Salt & Pepper Alto')
axs[1].axis('off')

## 4. Actividad

Aplique las anteriores estrategia de filtrado a imagenes contaminadas por ruido Salt & Pepper

---

In [ ]:
# aquí su código

---

## 5. Conclusiones

- <b>Efectividad de los filtros:</b> Los filtros pasabajas suavizan la imagen y reducen el ruido de alta frecuencia. Sin embargo, su efectividad varía dependiendo del tipo y densidad del ruido.

- <b>Ruido Salt & Pepper:</b> Este tipo de ruido es impulsivo y afecta píxeles individuales de manera extrema (negro a blanco o viceversa). Los filtros lineales como los de tipo box o Gaussianos no son muy efectivos para eliminar este tipo de ruido, especialmente en altas densidades, ya que tienden a difuminar el ruido en lugar de eliminarlo. Se recomienda filtros no lineales como por ejemplo el mediana.

- <b>Tamaño del kernel:</b> Un kernel más grande aumenta el efecto de suavizado, pero también puede causar una pérdida de detalles importantes en la imagen.